In [192]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

#File handle. Read the first one n lines from the Netflix movie list.

def read_movie(file, nrow):
    fhand = open(file)
    i = 0
    lines = []
    for line in fhand:
        i = i + 1
        lines.append(line.strip('\n\t').split(","))
        if i == nrow:
            break
    feature_names = ['ID', 'year', 'title']           
    lines = pd.DataFrame(lines)
    lines.columns = feature_names[:]
    return lines


# Generate movie search URL
def movie_link(movie_list):
    movie_url = []
    n_movie = len(movie_list['title'])
    for i in range(n_movie):
        title = movie_list['title'][i]
        title = title.replace(" ", "+")
        year = movie_list['year'][i]
        imdb_url = f'https://www.imdb.com/search/title/?title={title}&release_date={year},{year}&title_type=feature'
        movie_url.append(imdb_url)
    return movie_url

# Get genre
def genre(movie_url):
    movie_genre = []
    
    #move connection to main function
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    movie_info = soup.find("div", {"class": "lister-item mode-advanced"})
    try:
        movie_genre = movie_info.find("span",  {"class":  "genre"}).getText().replace('\n', '').strip()
    except:
        movie_genre = []
    return movie_genre

# Get cast
def cast(movie_url):
    cast = []

    #move connection to main function
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    movie_info = soup.find("div", {"class": "lister-item mode-advanced"})    
    try:
        cast = movie_info.find("p",  {"class":  ""}).getText().replace('\n', '').strip()
    except:
        cast = []
    return cast   


# Get rating
def rating(movie_url):
    length = []

    #move connection to main function
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    movie_info = soup.find("div", {"class": "lister-item mode-advanced"})    
    try:
        rating = movie_info.find("span",  {"class":  "certificate"}).getText().replace('\n', '').strip()
    except:
        rating = []
    return rating   

# Get length
def mlen(movie_url):
    length = []

    #move connection to main function
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    movie_info = soup.find("div", {"class": "lister-item mode-advanced"})    
    try:
        length = movie_info.find("span",  {"class":  "runtime"}).getText().replace('\n', '').strip()
    except:
        length = []
    return length  

# Main function
def get_info(movie):
    
    movie_url = movie_link(movie)
  
    get_genre = []
    get_cast = []
    get_rating = []
    get_mlen = []
    for i in movie_url:
        genre_tmp = genre(i)
        get_genre.append(genre_tmp)
        
        cast_tmp = cast(i)
        get_cast.append(cast_tmp)
        
        rating_tmp = rating(i)
        get_rating.append(rating_tmp)
        
        mlen_tmp = mlen(i)
        get_mlen.append(mlen_tmp)
        
    movie_detail = pd.DataFrame({
    "genre": get_genre,
    "cast": get_cast,
    "rating": get_rating,
    "length": get_mlen })
    movie_detail = pd.concat([movie, movie_detail], axis = 1)
    return movie_detail


In [196]:
#Test
movie = [['Batman', '1989' ], ['The Godfather', '1972'], ['The Matrix', '1999']]
#movie = [['Batman', '1989' ]]
df = pd.DataFrame(movie, columns = ['title', 'year'])
get_info(df)

,title,year,genre,cast,rating,length
0,Batman,1989,"Action, Adventure","Director:Tim Burton| Stars:Michael Keaton,...",PG-13,126 min
1,The Godfather,1972,"Crime, Drama",Director:Francis Ford Coppola| Stars:Marlo...,R,175 min
2,The Matrix,1999,"Action, Sci-Fi","Directors:Lana Wachowski, Lilly Wachowski| ...",R,136 min


In [187]:
# Use Netflix data
nf = read_movie("archive/movie_titles.csv", 40)

get_info(nf)

,ID,year,title,genre,cast,rating,length
0,1,2003,Dinosaur Planet,[],[],[],[]
1,2,2004,Isle of Man TT 2004 Review,[],[],[],[]
2,3,1997,Character,"Crime, Drama, Mystery",Director:Mike van Diem| Stars:Pavlik Janse...,R,122 min
3,4,1994,Paula Abdul's Get Up & Dance,[],[],[],[]
4,5,2004,The Rise and Fall of ECW,[],[],[],[]
5,6,1997,Sick,"Animation, Comedy",Director:Walter Santucci| Star:Jennifer Wi...,[],99 min
6,7,1992,8 Man,"Action, Sci-Fi",Director:Yasuhiro Horiuchi| Stars:Kai Shis...,[],83 min
7,8,2004,What the #$*! Do We Know!?,"Comedy, Fantasy, Horror","Stars:Kayvan Novak, Matt Berry, Natasia Demetr...",TV-MA,30 min
8,9,1991,Class of Nuke 'Em High 2,[],[],[],[]
9,10,2001,Fighter,[],[],[],[]
